# GEOL 593: Seismology and Earth Structure

## Lab assignment 5: Surface wave inversion

In today's lab we will explore how surface waves can be used to constrain the seismic velocity structure of the Earth's interior. In particular, we will learn how to measure **group velocity dispersion** of Rayleigh waves, and use a probabilistic inversion to estimate the seismic velocity structure in the crust and uppermost mantle. This is a particularly powerful method because it allows us to estimate a path-averaged seismic velocity profile using recordings of an earthquake at a single seismic station. 

The earthquake that we will analyze occurred on December 20, 2022 near the coast of Northern California, and resulted from strike-slip faulting. The hypocenter was located at a depth of 17.9 km and the event was a magnitude Mw 6.4. More information can be found on the IRIS event page here: https://ds.iris.edu/ds/nodes/dmc/tools/event/11635701. The event was clearly visible on broadband stations throughout the US. In this lab, we will look at waveforms from the station IU.DWPF, located in Disney Wilderness Preserve in Florida. By measuring Rayleigh wave group velocity dispersion from this event measured at IU.DWPF, we will be able to constrain the average shear wave speed structure between California and Florida.

### Package requirements

In addition to obspy and cartopy (which you have used previously), this lab requires two additional packages. The first package is `disba` (https://github.com/keurfonluu/disba), which will allow us to predict surface wave dispersion curves for different Earth models. The second package is `splipy` (https://pypi.org/project/Splipy/), which is a python library for dealing with B-spline functions. Both of these packages will be used for the probabilistic inversion of surface wave dispersion curves. They can both be installed with the package manager **pip** by typing the following in a terminal

`pip install disba` \
`pip install splipy`

If you have difficulty installing either of these packages, please let me know!


In [ ]:
#Imports
import disba
import cartopy
import numpy as np
import matplotlib.pyplot as plt
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from disba import GroupDispersion
from splipy import BSplineBasis
%matplotlib inline

### Downloading data

Run the cell below to download the data for the Mw 6.4 N. California event.

In [ ]:
#initialize IRIS client
client = Client("IRIS") 

#Indonesia event
origin = UTCDateTime('2022-12-20 10:34:24') #origin time of event
evlo = -124.42 #event longitude
evla = 40.525 #event latitude

starttime = origin
endtime = starttime + 60.*40 # we want 40 minutes of seismic data, starting at the origin time
st = client.get_waveforms("IU","DWPF", "00", "BHZ", starttime, endtime,attach_response=True) #download waveforms
st.remove_response(output='DISP',water_level=10.0)

### <font color='red'>Question 1 </font> 
After running the block above, you have the vertical component waveform data in *displacement* (in the Obspy stream `st`). Plot the displacement waveform using matplotlib. The x-axis should be in units of seconds and the y axis should be in units of meters.

In [ ]:
#Answer Q1 here.

### <font color='red'>Question 2 </font> 

The station IU.DWPF is located in the Disney Wilderness Preserve in Florida, with coordinates Latitude: 28.11, Longitude: -81.43. The earthquake (as shown above) has coordinates Latitude: 40.52, Longitude: -124.42. 

i) Use cartopy to plot a map of the earthquake and seismometer.

ii) Using the function `obspy.geodetics.gps2dist_azimuth`, calculate the epicentral distance of the event, in km.

Hint: The function `gps2dist_azimuth` takes 4 arguments; (lat1,lon1,lat2,lon2), where lat1 and lon1 are the coordinates of the earthquake and lat2 and lon2 are the coordinates of the seismic station. The function returns a python **list**. The first item of the list is the distance **in m**. See the documentation here: https://docs.obspy.org/packages/autogen/obspy.geodetics.base.gps2dist_azimuth.html

In [ ]:
#Answer Question 2 here.

### <font color='red'>Question 3 </font> 

Make a copy of your stream (e.g., `stc = st.copy()`), and bandpass filter it between 40 - 50 s (e.g., `stc.filter('bandpass',freqmin=1./50,freqmax=1/40.,corners=2,zerophase=True)`. Plot the filtered waveform, along with it's envelope. Hint; remember from lab 2 we can plot the envelope using the Hilbert transform; `scipy.signal.hilbert`)

At what time (in seconds) do you observe the peak of the envelope of the waveform? The time can be approximate (i.e., it's ok to eyeball it here). We will measure the dispersion more accurately next. Based on your calculation of event distance above, what group velocity does the peak of the envelope correspond to? Hint, the group velocity is calculated by dividing the distance of the event by the time that corresponds to the peak of the envelope function of the bandpass filtered signal.

In [ ]:
#Answer Q3 here.

### Measuring surface wave dispersion

The analysis you performed above is essentially how we measure surface wave group velocity. However, you performed the analysis using just one bandpass filter, so you only calculated one group velocity measurement (in this case for waves period between ~40 - 50 s). What we are most interested in with surface waves is the dependency of group velocity on frequency (i.e., the dispersion). To calculate the 'dispersion curve', we need to repeat the process above but for a range of frequency bands. Below, a function `calc_disp_curve` is provided to do this.

The function takes two arguments; i) the obspy data stream, and ii) the epicentral distance in km. If you run the function it will produce a Rayleigh wave group velocity dispersion plot. The plot shows period on the x-axis against group velocity (and time) on the y-axis. Each column of the plot is colored by the values of the envelope of the Rayleigh wave signal filtered at the corresponding period. The group velocity dispserion curve is found by finding the peak value (the brightest color) at each period. It can be thought of as following a 'ridge' of high values in the color plot. The measured dispersion curve is plotted as blue scatter points.

In [ ]:
def calc_dispersion(st,dist_km):
    '''
    creates a dispersion plot, provided a stream object (with only 1 trace), and an epicentral distance (in km)
    '''
    fig,ax = plt.subplots(figsize=[6,6])
    periods = np.linspace(15,50,36)
    time = st[0].times()
    disp_plot = []
    t_max = []
    
    for period in periods:
        period_min = period - 5.
        period_max = period + 5.
        freq_min = 1./period_max
        freq_max = 1./period_min
        stc = st.copy()
        stc.filter('bandpass',freqmin=freq_min,freqmax=freq_max,corners=4,zerophase=True)
        env = np.abs(hilbert(stc[0].data))
        env /= np.max(env)
        ind = np.argmax(env)
        t_max.append(time[ind])
        disp_plot.append(env)
    
    disp_plot = np.array(disp_plot)
    #plt.imshow(disp_plot,aspect='auto',cmap='magma')

    ax.pcolormesh(periods,time,disp_plot.T,cmap='magma')
    ax.set_ylim([1800,700])
    
    #functions for mapping time axis to group velocity axis
    def time_to_vel(x,dist_km=dist_km):
        vel = dist_km/x
        return vel
    def vel_to_time(x,dist_km=dist_km):
        time = dist_km/x
        return time
    
    #make secondary axis in group velocity
    gvel_ax = ax.secondary_yaxis('right', functions=(time_to_vel,vel_to_time))
    
    #convert t_max to group velocity
    t_max = np.array(t_max)
    group_vel = dist_km / t_max
    
    ax.set_xlabel('period (s)')
    ax.set_ylabel('time (s)')
    gvel_ax.set_ylabel('group velocity (km/s)')
    ax.scatter(periods,t_max)
    plt.show()

### <font color='red'>Question 4 </font> 

Modify the function above so that it returns both the group velocity dispersion curve and the periods at which they were measured. In other words, when you run the function, it should make the dispersion plot, and return two vectors which correspond to the scatter points shown in the plot.

Hint: When after you modify the function, calling the function should look something like this\
`per_observed, vel_observed = calc_dispserion(st,dist_km)` \
where `per_observed` and `vel_observed` are arrays corresponding to the periods and group velocities that were measured (i.e., the scatter points shown in the plot).

After modifying the function, run it below.

In [ ]:
#Answer Question 4 here.

### Inverting a dispersion curve for seismic velocity structure

The dispersion curve we just measured provides powerful constraints on the seismic velocity structure between the earthquake and the station. In the period range of 15 - 50 s, the waves are mostly sensitive to the crust and uppermost mantle. Our next step is to invert the dispersion curve for the average velocity-depth profile along the path of propagation. Unfortunately, because the 'sensitivity kernels' of surface waves (i.e., the regions in the subsurface to which surface waves are most sensitive) depend on the seismic wave speed structure we are trying to find, this problem is **non-linear**. In other words, we can not set this problem up as a matrix vector system $\mathbf{G}\mathbf{m} = \mathbf{d}$, as we have done with previous problems.

There are a variety of methods to solve non-linear problems, but here we are going to take a probabalistic approach based on *Bayes Theorem*. Bayes theorem relates the probability $p$ of a model $m$ given a dataset $d$ (written $p(m|d)$) to the probability of observing our dataset provided a model (written $p(d|m)$):

$p(m|d) \propto p(m)p(d|m)$

where $p(m|d)$ is called the *posterior* (the probability distribution we would like to find), $p(m)$ is called the *prior* (the constraints we have on the model from apriori information), and $p(d|m)$ is called the *likelihood function*. The likelihood function is defined as

$p(d|m) = \exp\left(\frac{-\phi(m)}{2}\right)$

where $\phi(m)$ is the misfit function based on the difference between our observations and the predictions for a given Earth model. We will use a least squares misfit function 

$\phi(m) = \lVert \frac{g(m) - d^2}{\sigma_d} \rVert$

Here, $g(m)$ corresponds to the *forward problem* (i.e., predicting the dispersion curve for a model $m$), $d$ is the observed dispersion curve, and $\sigma_d$ is the observational error (i.e., the standard error of the group velocity dispersion measurements). The data error $\sigma_d$ can be difficult to know exactly, but here we will assume a value of 0.1 km/s. 

Practically, to estimate the posterior probability disribution $p(m|d)$ we will probabilistically sample the *model space* (i.e., the range of plausible model parameters) using a *Markov-chain Monte Carlo* (MCMC) approach (https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo). Below, the function `run_mcmc` is given, which provides a crude MCMC algorithm. The aglorithm iteratively samples the model space by randomly selecting an Earth model (i.e., a seismic velocity structure) from a prior disribution. In our case, the prior distribution gives a range of reasonable seismic velocity values in the crust. The dispersion curve for this model is predicted (using the disba package), and the model is either accepted or rejected based on the misfit between the predicted and observed group velocity dispersion. If the model is accepted, it is added to a 'chain' of plausible models that fit the data reasonably well. After all MCMC iterations are complete, the chain of accepted models is used to approximate the posterior probability disribution $p(m|d)$. 

In the code block below, find two functions that are used to perform the inversion. The first function, `gen_model`, is a helper function to generate a model based on 'proposed' parameters (i.e., parameters drawn from our prior). The second function, `run_mcmc`, is used to run the MCMC inversion. There are three parameters that you must provide, detailed below:

    per_observed = an array of periods at which group velocity measurements were made (in s)
    vel_observed = an arary of group velocity measurements, made at each perion in 'per_observed'
    iterations = number of iterations to run the inversion for
    
Running `run_mcmc` will return three things:

    accepted = a list of accepted models
    misfits = a list of the misfits corresponding to the models in 'accepted'
    best_model = the model with the lowest misfit


In [ ]:
def gen_model(moho,coefs,vs_mantle):
    '''
    -------------------------------------------------------------------
    input parameters:
    -------------------------------------------------------------------
    moho =  the proposed moho depth (ie. crustal thickness), given in km
    coefs =  an array of proposed spline coefficients that describe the crustal shear velocity structure
    vs_mantle =  the proposed shear velocity in the mantle
    
    -------------------------------------------------------------------
    returns:
    -------------------------------------------------------------------
    vel_model = an array containing the velocity and density of the proposed model, in a format that 'disba' can read
    '''
    #fixed for 5 splines
    order = 3
    knots = np.array([0,0,0,0.35,0.7,1,1,1])
    
    #be careful changing the number of points. More layers means disba takes more time to calculate a dispersion curve.
    npts_crust = 20
    npts_mantle = 4
    
    crust = np.linspace(0,moho,npts_crust)
    mantle = np.linspace(moho,100,npts_mantle)
    depth = np.hstack((crust,mantle))

    knots *= np.max(crust)
    bs = BSplineBasis(knots=knots,order=order)
    basis = bs.evaluate(crust)

    nsplines = basis.shape[1]
    vs = np.zeros(npts_crust+npts_mantle)

    for i in range(0,nsplines):
        vs[0:npts_crust] += basis[:,i] * coefs[i]

    vs[npts_crust:] = vs_mantle

    thickness = np.diff(depth)[0]
    vs_layers = vs[0:len(vs)-1]
    vel_model = np.zeros((len(depth)-1,4))
            
    vel_model[:,0] = thickness              
    vel_model[:,1] = vs_layers * np.sqrt(3)  #Vp (km/s)
    vel_model[:,2] = vs_layers               #Vs (km/s)
    vel_model[:,3] = vs_layers / np.sqrt(2)  #density (gm/cm3)
    
    vel_model[-1,0] = 100.0 #extend the mantle thickness by 50 km.
    
    return vel_model

def run_mcmc(per_observed,vel_observed,iterations,burn_frac=0.2,sigma=0.1):
    '''
    -------------------------------------------------------------------
    input parameters:
    -------------------------------------------------------------------
    per_observed = an array of periods at which group velocity measurements were made (in s)
    vel_observed = an arary of group velocity measurements, made at each perion in 'per_observed'
    iterations = number of iterations to run the inversion for
    burn_frac = the percentage of the total iterations of the 'burn-in' period (these interations are thrown out)
    sigma = data error (defaults = 0.1 km/s)
    
    -------------------------------------------------------------------
    returns:
    -------------------------------------------------------------------
    accepted = a list of accepted models
    misfits = a list of the misfits corresponding to the models in 'accepted'
    best_model = the model with the lowest misfit
    '''
    #Set the priors on the model parameters. There are a total of 7 parameters. 
    #5 parameters describing Vs in the crust, a moho depth, and the Vs of the mantle (approximated as a single layer).
    p_low = [2.4,2.4,2.8,3.2,3.2]
    p_high = [3.2,3.2,3.6,4.0,4.0]
    moho_low = 10.0
    moho_high = 50.0
    vs_mantle_high = 4.8
    n_skipped = 0
    
    #initialize empty chain
    nchain = 0
    accepted = []
    misfits = []
    misfit_lowest = np.inf
    nburn = int(iterations*burn_frac)
    
    t = np.logspace(0.0,5.0,100)
    
    #run through the iterations
    for i in range(0,iterations):

        #randomly draw model parameters from uniform priors.  
        m0 = np.random.uniform(p_low[0],p_high[0])
        m1 = np.random.uniform(p_low[1],p_high[1])
        m2 = np.random.uniform(p_low[2],p_high[2])
        m3 = np.random.uniform(p_low[3],p_high[3])
        m4 = np.random.uniform(p_low[4],p_high[4])
        coefs = [m0,m1,m2,m3,m4]
        moho = np.random.uniform(moho_low,moho_high)
        vs_mantle = np.random.uniform(m4,vs_mantle_high)
        
        if i%100 == 0:
            print('iteration {}'.format(i))
            
        #run forward problem (ie. calculate a dispersion curve with disba)
        vel_model = gen_model(moho=moho,coefs=coefs,vs_mantle=vs_mantle)
        gd = GroupDispersion(*vel_model.T)
        
        try:
            gdr = gd(t,mode=0,wave='rayleigh')
        except disba.DispersionError:
            n_skipped += 1 #sometimes the forward calculation fails... just skip in this case
            continue

        per_modeled = gdr[0]
        vel_modeled = gdr[1]
        vel_modeled = np.interp(per_observed,per_modeled,vel_modeled) #interpolate modeled values
        
        #calculate misfit and likelihood function
        if i == 0:
            misfit0 = np.linalg.norm(((vel_observed - vel_modeled) / sigma)**2)
            phi0 = np.exp(-misfit0/2)
        else:
            misfit = np.linalg.norm(((vel_observed - vel_modeled) / sigma)**2)
            phi = np.exp(-misfit/2)
            r = phi / phi0
            
            if misfit < misfit_lowest and i > nburn:
                misfit_lowest = misfit
                best_model = [coefs,moho,vs_mantle]
                #print('misfit_lowest {}, best_model {}'.format(misfit_lowest,best_model))

            #roll a random number
            roll = np.random.uniform(0,1)

            if r > roll and i > nburn:
                nchain += 1
                accepted.append([coefs,moho,vs_mantle])
                misfits.append(misfit)
                
    print('{} skipped'.format(n_skipped))
    return accepted,misfits,best_model

### <font color='red'>Question 5 </font> 

Run the function `run_mcmc` to invert your measured group velocity dispersion curve, setting the number of iterations to 5000. The inversion will likely take several minutes. The function will print the current iteration number every 100 iterations, so you can monitor progress. If is taking a long time to run the inversion (e.g., it takes several minutes to perform 100 iterations), something is wrong and you should stop the process. Please let me know if this happens so I can help.

In general, the more iterations you run for, the more accurately your chain of accepted models will represent the true posterior probability distribution (until you get to a point of convergence). So you can try running the inversion with a different number of iterations and comparing the results, but you shouldn't perform too many iterations. I would avoid running it for many more than 10,000 because it will start to take a while.

In [ ]:
#Answer Question 5 here.

### Analyzing the results

Let's take a look at the inversion results! The function below, `plot_results`, will make a plot to summarize the inversion. The function takes 5 input arguments. The first three arguments (`accepted`, `misfits`, and `best_model`) are what was returned by `run_mcmc`. The remaining two arguments (`per_observed`, and `vel_observed`) describe the observed despersion curve (i.e., the periods and group velocity measurements, respectively). 

**Note:** Here we only plot the average of the best 100 models from the 'ensemble' of solutions that was returned by the MCMC inversion. In principle, the ensemble of models can be used to approximate the posterior probability distribution $p(m|d)$, which gives us information on the uncertainty of model parameters. To make things simple, we won't consider the model uncertainties, and only work with the mean model, and 'best' model.

In [ ]:
def plot_results(accepted, misfits, best_model, per_observed, vel_observed):
    '''
    plots a summary of the results
    '''
    
    fig,axes = plt.subplots(figsize=[10,5],ncols=2)
    
    #Generate 'best' model
    coefs_best = best_model[0]
    moho_best = best_model[1]
    vs_mantle_best = best_model[2]
    vel_best = gen_model(moho=moho_best,coefs=coefs_best,vs_mantle=vs_mantle_best)
    
    #Generate model from ensemble average (only use best N models): this is sort of cheating
    n_best = 100
    inds = np.argsort(misfits)

    accepted_nbest = []
    for i in range(0,n_best):
        accepted_nbest.append(accepted[inds[i]])

    coefs_accepted = 0
    moho_accepted = 0
    vs_mantle_accepted = 0

    for i in range(0,len(accepted_nbest)):
        coefs_accepted += np.array(accepted_nbest[i][0])
        moho_accepted += accepted_nbest[i][1]
        vs_mantle_accepted += accepted_nbest[i][2]

    ave_coefs = coefs_accepted/n_best
    ave_moho = moho_accepted/n_best
    ave_vs_mantle = vs_mantle_accepted/n_best
    vel_avg = gen_model(moho=ave_moho,coefs=ave_coefs,vs_mantle=ave_vs_mantle)
    
    #-----------------------------------------------------------------------------
    # plot models
    #-----------------------------------------------------------------------------
    axes[0].plot(vel_avg[:,2],np.cumsum(vel_avg[:,0]),label='mean model')
    axes[0].plot(vel_best[:,2],np.cumsum(vel_best[:,0]),label='best model')
    axes[0].legend()
    axes[0].set_xlim([1.5,5.5])
    axes[0].set_ylim([85,0])
    axes[0].set_xlabel('Vs (km/s)')
    axes[0].set_ylabel('depth (km)')
    
    #-----------------------------------------------------------------------------
    # plot observations vs model predictions
    #-----------------------------------------------------------------------------
    
    #observations
    axes[1].errorbar(per_observed,vel_observed,yerr=0.1,color='black',capsize=5,fmt='o',label='observed')

    #plot predictions for average model
    t = np.logspace(0.0,5.0,100)
    gd = GroupDispersion(*vel_avg.T)
    gdr = gd(t,mode=0,wave='rayleigh')
    vel_pred = np.interp(per_observed,gdr[0],gdr[1])
    axes[1].plot(per_observed,vel_pred,label='predicted, mean model',zorder=98)

    #plot predictions for best model
    gd = GroupDispersion(*vel_best.T)
    gdr = gd(t,mode=0,wave='rayleigh')
    vel_pred = np.interp(per_observed,gdr[0],gdr[1])
    axes[1].plot(per_observed,vel_pred,label='predicted, best model',zorder=99)
    
    axes[1].legend()
    axes[1].set_ylim([2.0,4.5])
    axes[1].set_xlabel('period (s)')
    axes[1].set_ylabel('group velocity (km/s)')
    plt.show()
    

### <font color='red'>Question 6 </font> 

i) Run `plot_results` to generate a figure. The figure will have two panels. The left panel shows the shear wave speed as a function of depth for i) the mean of the accepted models and ii) the best fitting model in the ensemble (i.e., the model with the lowest misfit). The right panel shows the observed group velocity dispersion measurements (with error bars depicting 0.1 km/s uncertainty bounds), along with the predicted dispersion curve for the mean model and best model.

ii) Describe the main features of the models, including the crustal thickness (i.e., Moho depth), and how Vs varies with depth.  

iii) Do the mean model and/or best model fit the data well? Why or why not?

In [ ]:
#Answer Question 6 here.

### <font color='red'>Question 7 </font> 

The CRUST2.0 model (https://igppweb.ucsd.edu/~gabi/crust2.html) gives a global estimate of crustal thickness at 2x2 degree resolution. Do your inversion results agree with what you would expect based on CRUST 2.0? Why or why not?

In [ ]:
#Answer Question 7 here.

### <font color='red'>Question 8 </font> 

In this analysis, we approximated the mantle as a single layer with a uniform velocity. 

i) Why is this a reasonable approximation? (or is it?)

ii) Let's say we want to resolve the velocity structure deeper in the mantle (e.g., down to 200 km) What could we change about this analysis to accomplish this? 

In [ ]:
# Answer Question 8 here